In [2]:
import requests

class URLMaker:
    """
    영화진흥위원회 API URL 생성 클래스
    """
    
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest'

    def __init__(self, key):
        """
        API key 초기화
        """
        self.key = key

    def daily_boxoffice_url(self, targetDt, itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''):
        """
        일별 박스오피스 URL 생성
        """
        return f'{self.url}/boxoffice/searchDailyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'

    def weekly_boxoffice_url(self, targetDt, weekGb='0', itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''):
        """
        주간/주말 박스오피스 URL 생성
        weekGb: 0(주간), 1(주말), 2(주중)
        """
        return f'{self.url}/boxoffice/searchWeeklyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&weekGb={weekGb}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'

    def movie_list_url(self, curPage=1, itemPerPage=10, movieNm='', directorNm='', openStartDt='', openEndDt='', 
                      prdtStartYear='', prdtEndYear='', repNationCd='', movieTypeCd=''):
        """
        영화목록 URL 생성
        """
        return f'{self.url}/movie/searchMovieList.json?key={self.key}&curPage={curPage}&itemPerPage={itemPerPage}&movieNm={movieNm}&directorNm={directorNm}&openStartDt={openStartDt}&openEndDt={openEndDt}&prdtStartYear={prdtStartYear}&prdtEndYear={prdtEndYear}&repNationCd={repNationCd}&movieTypeCd={movieTypeCd}'

    def movie_info_url(self, movieCd):
        """
        영화 상세정보 URL 생성
        """
        return f'{self.url}/movie/searchMovieInfo.json?key={self.key}&movieCd={movieCd}'

    def people_list_url(self, curPage=1, itemPerPage=10, peopleNm='', filmoNames=''):
        """
        영화인목록 URL 생성
        """
        return f'{self.url}/people/searchPeopleList.json?key={self.key}&curPage={curPage}&itemPerPage={itemPerPage}&peopleNm={peopleNm}&filmoNames={filmoNames}'

    def people_info_url(self, peopleCd):
        """
        영화인 상세정보 URL 생성
        """
        return f'{self.url}/people/searchPeopleInfo.json?key={self.key}&peopleCd={peopleCd}'

In [5]:
MY_API_KEY = 'ea85d0bf80c1a2a847737cf8270b6feb'
url_maker = URLMaker(MY_API_KEY)

## 영화 상세정보 조회

- **영화 상세정보 api**를 사용합니다.
- 영화 코드(movieCd)를 입력받아 해당 영화의 상세 정보를 조회합니다.
- 다음 정보를 순서대로 출력합니다:
    - 영화 제목 (제작연도)
    - 감독 (첫 번째 감독)
    - 주연 배우 (첫 번째, 두 번째 배우)
    - 장르 (첫 번째 장르)

In [20]:
movie_info_url = url_maker.movie_info_url('19950084')
print(movie_info_url)
# 이 구문이 함수 안으로 들어갔어야 했다
# '19950084'부분은 함수안으로 들어가면서 변수화 되어야 하겠다

http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=ea85d0bf80c1a2a847737cf8270b6feb&movieCd=19950084


In [8]:
def get_movie_detail(movie_cd):
    movie_url = url_maker.movie_info_url(movie_cd)
    movie_data = requests.get(movie_url).json()
    
    movie_info = movie_data['movieInfoResult']['movieInfo']
    
    print(f"{movie_info['movieNm']} ({movie_info['prdtYear']})")
    print(f"감독: {movie_info['directors'][0]['peopleNm']}")
    print(f"주연: {movie_info['actors'][0]['peopleNm']}, {movie_info['actors'][1]['peopleNm']}")
    print(f"장르: {movie_info['genres'][0]['genreNm']}")

In [19]:
get_movie_detail(input('영화코드(movieCd)를 입력해주세요 :'))

영화코드(movieCd)를 입력해주세요 : 19950084


쇼생크 탈출 (1994)
감독: 프랭크 다라본트
주연: 팀 로빈스, 모건 프리먼
장르: 드라마


## 일별 박스오피스 조회

- **일별 박스오피스 api**를 사용합니다.
- 입력받은 날짜(YYYYMMDD 형식)의 박스오피스 정보를 조회합니다.
- 각 영화의 순위, 영화제목, 해당 일의 관객수를 출력합니다.

### 받아온 url을 가공이 가능한 dict형태로 변경하는 것이 좋다
- 안그러면 데이터를 사용할 때마다 r.json을 매번 넣어줘야 한다
- ↓ bad case
```python
r = requests.get(daily_url)
daily_rank = r.json()['boxOfficeResult']['dailyBoxOfficeList']
print(daily_rank)
print(f'{daily_rank[0]['rank']}위 : {daily_rank[0]['movieNm']} ({daily_rank[0]['audiCnt']}명)')
```


In [59]:
def get_daily_boxoffice(target_date):
    daily_url = url_maker.daily_boxoffice_url(target_date)
    daily_data = requests.get(daily_url).json()
    # ↑ good case 받아온 url을 가공이 가능한 dict형태로 변경하는 것이 좋다
    
    boxoffice_list = daily_data['boxOfficeResult']['dailyBoxOfficeList']
    
    print(f"\n{target_date} 박스오피스 TOP 10")
    for movie in boxoffice_list:
        rank = movie['rank']
        title = movie['movieNm']
        audience = movie['audiCnt']
        print(f"{rank}위: {title} ({audience}명)")

In [57]:
get_daily_boxoffice(input('YYYYMMDD 형식으로 날짜 입력 :'))
# 20031111 부터 조회 가능

YYYYMMDD 형식으로 날짜 입력 : 20041123



20041123 박스오피스 TOP 10
1위: 나비효과 (28404명)
2위: 여선생 VS 여제자 (26918명)
3위: 내 머리 속의 지우개 (25705명)
4위: 이프 온리 (14070명)
5위: 하나와 앨리스 (5655명)
6위: 엑소시스트: 더 비기닝 (5490명)
7위: 화이트 칙스 (4173명)
8위: 레지던트 이블 2-아포칼립스 (4140명)
9위: 팜므 파탈 (2639명)
10위: 주홍글씨 (2543명)


## 한국 영화 일별 박스오피스 조회

- **일별 박스오피스 api**를 사용합니다.
- 입력받은 날짜(YYYYMMDD 형식)의 한국 영화 박스오피스 정보를 조회합니다.
- 한국 영화만 조회하기 위해 repNationCd='K' 파라미터를 사용합니다.
- 영화가 개봉한 순서대로 정렬하여 제목, 날짜, 순위를 출력합니다.

In [65]:
# version1 (리스트에 저장 후 정렬하는 방식)
def get_movie_details(target_date):
    daily_url = url_maker.daily_boxoffice_url(targetDt=target_date, repNationCd='K')
    daily_data = requests.get(daily_url).json()
    movies = daily_data['boxOfficeResult']['dailyBoxOfficeList']

    movie_list = []
    for movie in movies:
        movie_list.append([movie['openDt'], movie['rank'], movie['movieNm']])
    print(movie_list)
get_movie_details('20250201')
# 날짜데이터를 리스트안에서 맨 앞에 둠으로써 정렬할때 우선순위를 갖게 한다

[['2025-01-22', '1', '히트맨2'], ['2025-01-24', '2', '검은 수녀들'], ['2025-01-27', '3', '말할 수 없는 비밀'], ['2024-12-24', '4', '하얼빈'], ['2025-01-01', '5', '뽀로로 극장판 바닷속 대모험'], ['2025-01-24', '6', '아이유 콘서트 : 더 위닝'], ['2025-01-24', '7', '귀신경찰'], ['2024-12-04', '8', '소방관'], ['2011-07-28', '9', '마당을 나온 암탉'], [' ', '10', '뜻밖의 순간 : 언익스펙티드 저니']]


In [67]:
# sorted 먹여서 날짜순으로 정렬된 모습
def get_movie_details(target_date):
    daily_url = url_maker.daily_boxoffice_url(targetDt=target_date, repNationCd='K')
    daily_data = requests.get(daily_url).json()
    movies = daily_data['boxOfficeResult']['dailyBoxOfficeList']

    movie_list = []
    for movie in movies:
        movie_list.append([movie['openDt'], movie['rank'], movie['movieNm']])

    sorted_movies = sorted(movie_list)
    print(sorted_movies)
get_movie_details('20250201')

[[' ', '10', '뜻밖의 순간 : 언익스펙티드 저니'], ['2011-07-28', '9', '마당을 나온 암탉'], ['2024-12-04', '8', '소방관'], ['2024-12-24', '4', '하얼빈'], ['2025-01-01', '5', '뽀로로 극장판 바닷속 대모험'], ['2025-01-22', '1', '히트맨2'], ['2025-01-24', '2', '검은 수녀들'], ['2025-01-24', '6', '아이유 콘서트 : 더 위닝'], ['2025-01-24', '7', '귀신경찰'], ['2025-01-27', '3', '말할 수 없는 비밀']]


```python
movie[0] → 개봉일 (openDt)
movie[1] → 순위 (rank)
movie[2] → 영화 제목 (movieNm)
```
이므로 최종버전은 아래와 같다

In [68]:
def get_movie_details(target_date):
    daily_url = url_maker.daily_boxoffice_url(targetDt=target_date, repNationCd='K')
    daily_data = requests.get(daily_url).json()
    movies = daily_data['boxOfficeResult']['dailyBoxOfficeList']

    movie_list = []
    for movie in movies:
        movie_list.append([movie['openDt'], movie['rank'], movie['movieNm']])

    sorted_movies = sorted(movie_list)
    print(f"{target_date} 한국 영화 박스오피스 (개봉일 순)")
    for movie in sorted_movies:
        print(f"{movie[2]} ({movie[0]}) - {movie[1]}위")

### 이제까지 함수에서 지속적으로 **반복되어 나오는 구조**
```python
def get_movie_details(target_date):
    daily_url = url_maker.daily_boxoffice_url(targetDt=target_date, repNationCd='K')
    daily_data = requests.get(daily_url).json()
    movies = daily_data['boxOfficeResult']['dailyBoxOfficeList']

    movie_list = []
    for movie in movies:
        pass
```
그리고 `for` 안에서 무엇을 반복하며 리스트안에 넣고 싶은가만 달라지는 것이었다
1~3번 문제까지 오면서 이 구조를 머릿속에서 잡지못해서 문제를 어렵게 풀었다

## 1월 전체 주간 박스오피스 데이터 수집

- **주간/주말 박스오피스 api**를 사용합니다.
- 2025년 1월의 모든 주간(1/5, 1/12, 1/19, 1/26) 데이터를 수집합니다.
- 각 영화의 정보는 다음을 포함해야 합니다:
    - 영화제목, 개봉일, 누적관객수, 누적매출액
- 동일한 영화가 여러 주에 걸쳐 나타날 경우, 가장 최근의 정보만 저장합니다.
```python
 def weekly_boxoffice_url(self, targetDt, weekGb='0', itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''):
        """
        주간/주말 박스오피스 URL 생성
        weekGb: 0(주간), 1(주말), 2(주중)
        """
        return f'{self.url}/boxoffice/searchWeeklyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&weekGb={weekGb}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'
```


In [74]:
# 그리고 함수 구조 짜나갈 때, def밖에서 짜보고 나중에 def안으로 집어넣는 방식으로 했는데 잘못된 방법같다
# 나중에 def안으로 넣을때 indent하기 너무 귗ㅏㄴㅎ다...
def get_weekly_boxoffice():
    jan = ['20250105', '20250112', '20250119', '20250126']
    # 위의 부분을 from datetime import datetime 해서도 풀 수 있는지가 궁금하다 나중에 공부하게되면 추가해본다
    movie_data = {} # 출력예시가 {k: v}인 dict 구조였다
    
    for target_date in jan:                       # '주간'이었던걸 완전 잊고있었다
        weekly_url = url_maker.weekly_boxoffice_url(target_date, weekGb='0')
        weekly_data = requests.get(weekly_url).json()
        
        boxoffice_list = weekly_data['boxOfficeResult']['weeklyBoxOfficeList']
        showRange = weekly_data['boxOfficeResult']['showRange']
        # 주차별 자료 안에서 for문 돌린다
        for movie in boxoffice_list:
            movie_data[movie['movieNm']] = {
                '영화제목': movie['movieNm'],
                '개봉일': movie['openDt'],
                '누적관객': movie['audiAcc'],
                '누적매출': movie['salesAcc'],
            }
    
    print(movie_data)

In [75]:
get_weekly_boxoffice()

{'하얼빈': {'영화제목': '하얼빈', '개봉일': '2024-12-24', '누적관객': '4642215', '누적매출': '44407362460'}, '소방관': {'영화제목': '소방관', '개봉일': '2024-12-04', '누적관객': '3840551', '누적매출': '36145572270'}, '보고타: 마지막 기회의 땅': {'영화제목': '보고타: 마지막 기회의 땅', '개봉일': '2024-12-31', '누적관객': '396287', '누적매출': '3669720540'}, '수퍼 소닉3': {'영화제목': '수퍼 소닉3', '개봉일': '2025-01-01', '누적관객': '322100', '누적매출': '2950620910'}, '뽀로로 극장판 바닷속 대모험': {'영화제목': '뽀로로 극장판 바닷속 대모험', '개봉일': '2025-01-01', '누적관객': '349597', '누적매출': '2996015770'}, '무파사: 라이온 킹': {'영화제목': '무파사: 라이온 킹', '개봉일': '2024-12-18', '누적관객': '833832', '누적매출': '8188072950'}, '모아나 2': {'영화제목': '모아나 2', '개봉일': '2024-11-27', '누적관객': '3522517', '누적매출': '32533317750'}, '극장판 짱구는 못말려: 우리들의 공룡일기': {'영화제목': '극장판 짱구는 못말려: 우리들의 공룡일기', '개봉일': '2024-12-18', '누적관객': '762895', '누적매출': '7177275100'}, '위키드': {'영화제목': '위키드', '개봉일': '2024-11-20', '누적관객': '2195165', '누적매출': '21677201940'}, '시빌 워: 분열의 시대': {'영화제목': '시빌 워: 분열의 시대', '개봉일': '2024-12-31', '누적관객': '80373', '누적매출': '836845230'}, '동화지만 청불입니다': {'영

## 일별 박스오피스 영화의 상세정보

- **일별 박스오피스 api**와 **영화 상세정보 api**를 사용합니다.
- 입력받은 날짜의 일별 박스오피스 TOP 10을 조회합니다.
- 각 영화의 상세 정보를 추가로 조회하여 다음 정보를 출력합니다:
    - 순위
    - 영화제목
    - 감독
    - 주연배우 (최대 3명)

In [76]:
def get_movie_details(target_date):
    pass

In [77]:
get_movie_details("20250201")  # YYYYMMDD 형식으로 날짜 입력

**출력예시**

```
1위
히트맨2
감독: 최원섭
주연: 권상우, 정준호, 이이경
----------------------------------------

2위
검은 수녀들
감독: 권혁재
주연: 송혜교, 전여빈, 이진욱
----------------------------------------

3위
말할 수 없는 비밀
감독: 서유민
주연: 도경수, 원진아, 신예은
----------------------------------------
...
```

## 배우검색

- **영화인목록 api**, **영화인 상세정보 api**, **영화 상세정보 api**를 사용합니다.
- 배우 이름을 입력받아 해당 배우의 정보를 검색합니다.
- 여러 명의 동명 배우가 있는 경우, 필모그래피가 가장 많은 배우를 선택합니다.
- 선택된 배우의 출연작 중 '배우'로 참여한 작품 5개의 정보를 출력합니다:
    - 영화 제목
    - 개봉일
    - 상영시간
    - 감독

In [79]:
def analyze_actor_movies(actor_name):
    pass

In [80]:
analyze_actor_movies("박정민")  # 원하는 배우 이름으로 변경 가능

**출력예시**

```
배우 '박정민'의 최근 출연작 5개

제목: 하얼빈
개봉일: 20241224
상영시간: 113분
감독: 우민호

제목: 얼굴
개봉일: 
상영시간: 분
감독: 연상호

제목: 전, 란
개봉일: 
상영시간: 126분
감독: 김상만

제목: 휴민트
개봉일: 
상영시간: 분
감독: 류승완

제목: 밀수
개봉일: 20230726
상영시간: 129분
감독: 류승완
```